In [1]:
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torch.cuda import FloatTensor, LongTensor

np.random.seed(42)

In [2]:
from utils import *
#from model import build_decoder_model

%load_ext autoreload
%autoreload 2

c:\python36\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
vocab = Vocab("finished_files/vocab", VOCAB_SIZE)









max_size of vocab was specified as 50000; we now have 50000 words. Stopping reading.
Finished constructing vocabulary of 50000 total words. Last word added: perisic


In [4]:
model = build_decoder_model()
torch.cuda.memory_allocated()

135609856

train_data=Batcher(vocab, "finished_files/chunked/train_000.bin", 8, 1000, 50)

gen_and_print_summary(train_data, model, 10)

In [5]:
criterion = nn.CrossEntropyLoss(ignore_index = vocab.word2id(PAD_TOKEN)).cuda()
optimizer = optim.Adagrad(model.parameters(), lr=0.15, initial_accumulator_value=0.1)

In [6]:
#model.load_model("cov_loss3")

In [7]:
#all_data = Batcher(vocab, "finished_files/train.bin",32, 200, 50)

In [8]:
#dta = Batcher(vocab, "finished_files/chunked/train_001.bin",32, 200, 100)

In [9]:
fit(model, criterion, optimizer, epochs_count=2, batch_size=32, train_data=Batcher(vocab, "finished_files/train.bin",64, 100, 10),
    val_data=None, val_batch_size=32, cov_loss=False)
model.save_model("loss2")
fit(model, criterion, optimizer, epochs_count=2, batch_size=32, train_data=Batcher(vocab, "finished_files/train.bin",64, 100, 20),
    val_data=None, val_batch_size=32, cov_loss=False)
model.save_model("loss2")
fit(model, criterion, optimizer, epochs_count=5, batch_size=32, train_data=Batcher(vocab, "finished_files/train.bin",32, 200, 50),
    val_data=None, val_batch_size=32, cov_loss=False)
model.save_model("loss2")
fit(model, criterion, optimizer, epochs_count=5, batch_size=32, train_data=Batcher(vocab, "finished_files/train.bin",32, 200, 50),
    val_data=None, val_batch_size=32, cov_loss=True)
model.save_model("loss2")
fit(model, criterion, optimizer, epochs_count=5, batch_size=32, train_data=Batcher(vocab, "finished_files/train.bin",16, 400, 100),
    val_data=None, val_batch_size=32, cov_loss=False)
model.save_model("loss2")

4488
377


c:\python36\lib\site-packages\nltk\translate\bleu_score.py:503: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
c:\python36\lib\site-packages\nltk\translate\bleu_score.py:503: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
c:\python36\lib\site-packages\nltk\translate\bleu_score.py:503: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)


Epoch 1 / 2, Epoch Time = 609.44s: Train Loss = 197586.2619: Cov_Loss = 0.0000, BLEU = 0.1825, ROUGE = 0.1825
Epoch 2 / 2, Epoch Time = 611.17s: Train Loss = 171138.8380: Cov_Loss = 0.0000, BLEU = 0.2562, ROUGE = 0.2562

4488
377
Epoch 1 / 2, Epoch Time = 1041.79s: Train Loss = 363796.6244: Cov_Loss = 0.0000, BLEU = 0.2530, ROUGE = 0.2532
Epoch 2 / 2, Epoch Time = 1041.06s: Train Loss = 349065.8202: Cov_Loss = 0.0000, BLEU = 0.2652, ROUGE = 0.2652

8976
377
Epoch 1 / 5, Epoch Time = 3872.48s: Train Loss = 1587626.5990: Cov_Loss = 0.0000, BLEU = 0.2922, ROUGE = 0.2929
Epoch 2 / 5, Epoch Time = 3849.63s: Train Loss = 1536621.1641: Cov_Loss = 0.0000, BLEU = 0.3018, ROUGE = 0.3022
Epoch 3 / 5, Epoch Time = 3844.24s: Train Loss = 1505625.5562: Cov_Loss = 0.0000, BLEU = 0.3064, ROUGE = 0.3068
Epoch 4 / 5, Epoch Time = 3847.43s: Train Loss = 1482198.0046: Cov_Loss = 0.0000, BLEU = 0.3099, ROUGE = 0.3103
Epoch 5 / 5, Epoch Time = 3858.82s: Train Loss = 1463383.5874: Cov_Loss = 0.0000, BLEU = 0

In [12]:
fit(model, criterion, optimizer, epochs_count=5, batch_size=32, train_data=Batcher(vocab, "finished_files/train.bin",16, 400, 100),
    val_data=None, val_batch_size=32, cov_loss=True)
model.save_model("loss22")

17952
377
[0]: Loss = 177.1675, Cov_Loss = 0.7176, BLEU = 0.3497, ROUGE = 0.3514

c:\python36\lib\site-packages\nltk\translate\bleu_score.py:503: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
c:\python36\lib\site-packages\nltk\translate\bleu_score.py:503: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)


[5]: Loss = 194.4483, Cov_Loss = 0.6834, BLEU = 0.3167, ROUGE = 0.3183

c:\python36\lib\site-packages\nltk\translate\bleu_score.py:503: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)


Epoch 1 / 5, Epoch Time = 12521.85s: Train Loss = 3285087.5947: Cov_Loss = 12371.0177, BLEU = 0.3506, ROUGE = 0.3530
Epoch 2 / 5, Epoch Time = 12520.84s: Train Loss = 3266359.6590: Cov_Loss = 12347.4629, BLEU = 0.3519, ROUGE = 0.3544
Epoch 3 / 5, Epoch Time = 12611.67s: Train Loss = 3249385.1384: Cov_Loss = 12326.2226, BLEU = 0.3530, ROUGE = 0.3554
Epoch 4 / 5, Epoch Time = 12622.93s: Train Loss = 3233252.2001: Cov_Loss = 12304.3834, BLEU = 0.3541, ROUGE = 0.3566
Epoch 5 / 5, Epoch Time = 12617.87s: Train Loss = 3218176.7367: Cov_Loss = 12284.4438, BLEU = 0.3552, ROUGE = 0.3576



In [13]:
fit(model, criterion, optimizer, epochs_count=20, batch_size=32, train_data=Batcher(vocab, "finished_files/train.bin",32, 200, 20),
    val_data=None, val_batch_size=32, cov_loss=False)
model.save_model("loss2")

8976
377


c:\python36\lib\site-packages\nltk\translate\bleu_score.py:503: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
c:\python36\lib\site-packages\nltk\translate\bleu_score.py:503: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
c:\python36\lib\site-packages\nltk\translate\bleu_score.py:503: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)


Epoch 1 / 20, Epoch Time = 1758.52s: Train Loss = 512089.5733: Cov_Loss = 0.0000, BLEU = 0.3450, ROUGE = 0.3452
[6207]: Loss = 53.3432, Cov_Loss = 0.0000, BLEU = 0.4170, ROUGE = 0.4170

KeyboardInterrupt: 

fit(model, criterion, optimizer, epochs_count=10, batch_size=32, train_data=all_data,
    val_data=None, val_batch_size=32, cov_loss=True)

In [10]:
#model.save_model("cov_loss3")

fit(model, criterion, optimizer, epochs_count=20, batch_size=32, train_data=all_data,
    val_data=None, val_batch_size=32, cov_loss=True)

In [11]:
#model.save_model("cov_loss3")

max_len = 400
max_tg_len = 100
art_len = 60
tg_len = 30


for i in range(18):
    batch_size = int(64 / (i + 1))
    art_len += 20
    tg_len += 10
    if tg_len > max_tg_len:
        tg_len = max_tg_len
    if art_len > max_len:
        art_len = max_len
    fit(model, criterion, optimizer, epochs_count=5, batch_size=32, train_data=Batcher(vocab, "finished_files/train.bin",batch_size, art_len, tg_len),
    val_data=None, val_batch_size=32)

fit(model, criterion, optimizer, epochs_count=20, batch_size=32, train_data=all_data,
    val_data=None, val_batch_size=32, cov_loss=True)